In [157]:
import pandas as pd
import numpy as np

# Carga de los datasets
ruta_base = '../data/raw/'

## DATASET DE PRODUCCION POR SECTOR INDUSTRIAL

In [158]:
archivo = ruta_base + '14_3_03_Produccion_Industrial-1-1.xlsx'

hojas_y_skips = {
    'CONFECCIONISTA': 6,
    'ELECTRONICA': 6,
    'PESQUERA': 6,
    'TEXTIL': 6,
    'PLASTICA': 6,
    'OTROS': 5
}

lista_df = []

for hoja, skip in hojas_y_skips.items():
    print(f"\n📄 Leyendo hoja: {hoja} (skiprows={skip})")
    df_temp = pd.read_excel(archivo, sheet_name=hoja, skiprows=skip)

    try:
        if hoja == 'OTROS':
            df_temp = df_temp.iloc[:, [0, 1, 4]]  # Año, Mes, Producción (en columna E)
        else:
            df_temp = df_temp.iloc[:, [0, 1, 2]]  # Año, Mes, Producción (columna C)
        
        df_temp.columns = ['anio', 'mes', 'Produccion']
        df_temp['anio'] = df_temp['anio'].fillna(method='ffill')
        df_temp = df_temp.dropna(subset=['anio', 'mes', 'Produccion'])
        df_temp['anio'] = df_temp['anio'].astype(int)
        df_temp['mes'] = df_temp['mes'].astype(str)
        df_temp['sector'] = hoja.capitalize()
        lista_df.append(df_temp)
    except Exception as e:
        print(f"❌ Error en hoja {hoja}: {e}")

# Unimos todos los sectores
df_produccion_total = pd.concat(lista_df, ignore_index=True)

# Reordenamos columnas
df_produccion_total = df_produccion_total[['anio', 'mes', 'sector', 'Produccion']]

# Vista final
print("✅ Producción final normalizada y completa:")
display(df_produccion_total.head(15))



📄 Leyendo hoja: CONFECCIONISTA (skiprows=6)


C:\Users\Luciano\AppData\Local\Temp\ipykernel_4868\1298280039.py:25: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_temp['anio'] = df_temp['anio'].fillna(method='ffill')



📄 Leyendo hoja: ELECTRONICA (skiprows=6)


C:\Users\Luciano\AppData\Local\Temp\ipykernel_4868\1298280039.py:25: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_temp['anio'] = df_temp['anio'].fillna(method='ffill')
C:\Users\Luciano\AppData\Local\Temp\ipykernel_4868\1298280039.py:25: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_temp['anio'] = df_temp['anio'].fillna(method='ffill')



📄 Leyendo hoja: PESQUERA (skiprows=6)

📄 Leyendo hoja: TEXTIL (skiprows=6)


C:\Users\Luciano\AppData\Local\Temp\ipykernel_4868\1298280039.py:25: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_temp['anio'] = df_temp['anio'].fillna(method='ffill')



📄 Leyendo hoja: PLASTICA (skiprows=6)

📄 Leyendo hoja: OTROS (skiprows=5)
✅ Producción final normalizada y completa:


C:\Users\Luciano\AppData\Local\Temp\ipykernel_4868\1298280039.py:25: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_temp['anio'] = df_temp['anio'].fillna(method='ffill')
C:\Users\Luciano\AppData\Local\Temp\ipykernel_4868\1298280039.py:25: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_temp['anio'] = df_temp['anio'].fillna(method='ffill')


,anio,mes,sector,Produccion
0,2014,Ene,Confeccionista,243751.0
1,2014,Feb,Confeccionista,386730.0
2,2014,Mar,Confeccionista,413565.0
3,2014,Apr,Confeccionista,446065.0
4,2014,May,Confeccionista,547531.0
5,2014,Jun,Confeccionista,450209.0
6,2014,Jul,Confeccionista,350009.0
7,2014,Ago,Confeccionista,365418.0
8,2014,Sep,Confeccionista,419830.0
9,2014,Oct,Confeccionista,462323.0


In [159]:
df_produccion_total['sector'].value_counts()

sector
Confeccionista    135
Electronica       135
Pesquera          135
Textil            135
Otros             135
Plastica          134
Name: count, dtype: int64

In [160]:
df_produccion_total.tail(10)

,anio,mes,sector,Produccion
799,2024,Jun*,Otros,8291.0
800,2024,Jul*,Otros,10536.0
801,2024,Ago*,Otros,11832.0
802,2024,Sep*,Otros,10740.0
803,2024,Oct*,Otros,12672.0
804,2024,Nov*,Otros,11712.0
805,2024,Dic*,Otros,5400.0
806,2025,Ene*,Otros,6600.0
807,2025,Feb*,Otros,9672.0
808,2025,Mar*,Otros,8844.0


In [161]:
df_produccion_total[df_produccion_total['sector'] == 'Plastica']['anio'].value_counts()


anio
2014    12
2015    12
2016    12
2017    12
2018    12
2019    12
2020    12
2021    12
2022    12
2023    12
2024    12
2025     2
Name: count, dtype: int64

In [162]:
df_produccion_total

,anio,mes,sector,Produccion
0,2014,Ene,Confeccionista,243751.0
1,2014,Feb,Confeccionista,386730.0
2,2014,Mar,Confeccionista,413565.0
3,2014,Apr,Confeccionista,446065.0
4,2014,May,Confeccionista,547531.0
...,...,...,...,...
804,2024,Nov*,Otros,11712.0
805,2024,Dic*,Otros,5400.0
806,2025,Ene*,Otros,6600.0
807,2025,Feb*,Otros,9672.0


In [164]:
# Verificar los valores únicos de la columna 'mes' en producción
print(sorted(df_produccion_total['mes'].unique()))

['Abr*', 'Ago', 'Ago ', 'Ago*', 'Apr', 'Apr ', 'Dic', 'Dic ', 'Dic*', 'Ene', 'Ene ', 'Ene*', 'Feb', 'Feb ', 'Feb*', 'Jul', 'Jul ', 'Jul*', 'Jun', 'Jun ', 'Jun*', 'Mar', 'Mar ', 'Mar*', 'May', 'May ', 'May*', 'Nov', 'Nov ', 'Nov*', 'Oct', 'Oct ', 'Oct*', 'Sep', 'Sep ', 'Sep*']


In [165]:
# Diccionario base para normalizar abreviaciones o errores
meses_validos = {
    'ene': 'enero', 'feb': 'febrero', 'mar': 'marzo',
    'abr': 'abril', 'may': 'mayo', 'jun': 'junio',
    'jul': 'julio', 'ago': 'agosto', 'sep': 'septiembre',
    'oct': 'octubre', 'nov': 'noviembre', 'dic': 'diciembre', 'apr':'abril'
}

# Limpieza unificada
df_produccion_total['mes'] = (
    df_produccion_total['mes']
    .astype(str)
    .str.lower()
    .str.strip()
    .str.replace(r'\*', '', regex=True)
    .str.replace(r'\s+', '', regex=True)
    .map(lambda x: next((v for k, v in meses_validos.items() if x.startswith(k)), x))
)


In [166]:
# Verificar los valores únicos de la columna 'mes' en producción
print(sorted(df_produccion_total['mes'].unique()))

['abril', 'agosto', 'diciembre', 'enero', 'febrero', 'julio', 'junio', 'marzo', 'mayo', 'noviembre', 'octubre', 'septiembre']


In [167]:
print(sorted(df_produccion_total['sector'].unique()))

['Confeccionista', 'Electronica', 'Otros', 'Pesquera', 'Plastica', 'Textil']


In [168]:
print("📌 Tipos de datos en df_produccion_total:")
print(df_produccion_total.dtypes)

print("\n📌 Tipos reales en columna 'sector':")
print(df_produccion_total['sector'].apply(type).value_counts())


📌 Tipos de datos en df_produccion_total:
anio           int32
mes           object
sector        object
Produccion    object
dtype: object

📌 Tipos reales en columna 'sector':
sector
<class 'str'>    809
Name: count, dtype: int64


## DATASET DE EMPLEADOS POR SECTOR INDUSTRIAL

In [169]:
# Cargar archivo
archivo_empleados = ruta_base + '14_3_01_Personal_industria_rama-1.xlsx'
df_emp = pd.read_excel(archivo_empleados, skiprows=2)

# Cortamos solo las columnas necesarias (A → H = 0:8)
df_emp = df_emp.iloc[:, 0:8]
df_emp.columns = ['anio', 'mes', 'Confeccionista', 'Electrónicas', 'Plásticas', 'Textiles', 'Pesqueras', 'Otras']

# Rellenamos año hacia abajo
df_emp['anio'] = df_emp['anio'].ffill()

# Quitamos filas vacías de año o mes
df_emp = df_emp.dropna(subset=['anio', 'mes'])
df_emp['anio'] = df_emp['anio'].astype(int)
df_emp['mes'] = df_emp['mes'].astype(str)

# Pasamos a formato largo
df_empleados_limpio = df_emp.melt(
    id_vars=['anio', 'mes'],
    var_name='sector',
    value_name='empleados'
)

# Capitalizamos sector
df_empleados_limpio['sector'] = df_empleados_limpio['sector'].str.capitalize()

# Mostramos
print("✅ Dataset de empleados normalizado:")
display(df_empleados_limpio.head(15))


✅ Dataset de empleados normalizado:


,anio,mes,sector,empleados
0,2001,Enero,Confeccionista,326.0
1,2001,Febrero,Confeccionista,323.0
2,2001,Marzo,Confeccionista,333.0
3,2001,Abril,Confeccionista,337.0
4,2001,Mayo,Confeccionista,346.0
5,2001,Junio,Confeccionista,352.0
6,2001,Julio,Confeccionista,354.0
7,2001,Agosto,Confeccionista,351.0
8,2001,Septiembre,Confeccionista,347.0
9,2001,Octubre,Confeccionista,339.0


In [170]:
# Cuántos valores nulos hay por sector
df_empleados_limpio.isna().groupby(df_empleados_limpio['sector']).sum()

,anio,mes,sector,empleados
sector,,,,
Confeccionista,0,0,0,0
Electrónicas,0,0,0,0
Otras,0,0,0,0
Pesqueras,0,0,0,0
Plásticas,0,0,0,0
Textiles,0,0,0,0


In [171]:
# Conteo de filas por sector
df_empleados_limpio['sector'].value_counts()

sector
Confeccionista    291
Electrónicas      291
Plásticas         291
Textiles          291
Pesqueras         291
Otras             291
Name: count, dtype: int64

In [172]:
df_empleados_limpio[
    df_empleados_limpio['sector'] == 'Pesqueras'].head(30)


,anio,mes,sector,empleados
1164,2001,Enero,Pesqueras,353
1165,2001,Febrero,Pesqueras,278
1166,2001,Marzo,Pesqueras,275
1167,2001,Abril,Pesqueras,273
1168,2001,Mayo,Pesqueras,262
1169,2001,Junio,Pesqueras,259
1170,2001,Julio,Pesqueras,166
1171,2001,Agosto,Pesqueras,166
1172,2001,Septiembre,Pesqueras,188
1173,2001,Octubre,Pesqueras,187


In [173]:
# Convertimos guiones y strings no numéricos a NaN en 'empleados'
df_empleados_limpio['empleados'] = pd.to_numeric(df_empleados_limpio['empleados'], errors='coerce')

In [174]:
# Cuántos valores nulos hay por sector
df_empleados_limpio.isna().groupby(df_empleados_limpio['sector']).sum()

,anio,mes,sector,empleados
sector,,,,
Confeccionista,0,0,0,0
Electrónicas,0,0,0,0
Otras,0,0,0,0
Pesqueras,0,0,0,3
Plásticas,0,0,0,0
Textiles,0,0,0,0


In [175]:
# Normalizar los valores de 'mes': minúsculas, sin espacios ni asteriscos
# Diccionario de normalización (corrige también abreviaturas y nombres mal escritos)
mes_completo = {
    'ene': 'enero', 'enero': 'enero',
    'feb': 'febrero', 'febrero': 'febrero',
    'mar': 'marzo', 'marzo': 'marzo',
    'abr': 'abril', 'abril': 'abril',
    'may': 'mayo', 'mayo': 'mayo',
    'jun': 'junio', 'junio': 'junio',
    'jul': 'julio', 'julio': 'julio',
    'ago': 'agosto', 'agosto': 'agosto',
    'sep': 'septiembre', 'septiembre': 'septiembre',
    'oct': 'octubre', 'octubre': 'octubre',
    'nov': 'noviembre', 'noviembre': 'noviembre',
    'dic': 'diciembre', 'diciembre': 'diciembre'
}

# Limpieza completa
df_empleados_limpio['mes'] = (
    df_empleados_limpio['mes']
    .astype(str)
    .str.strip()
    .str.lower()
    .str.replace(r'\*', '', regex=True)
    .map(mes_completo)  # ← reemplaza por nombres completos
)


In [176]:
# Verificamos valores únicos
print(sorted(df_empleados_limpio['mes'].unique()))

['abril', 'agosto', 'diciembre', 'enero', 'febrero', 'julio', 'junio', 'marzo', 'mayo', 'noviembre', 'octubre', 'septiembre']


In [177]:
df_empleados_limpio[
    df_empleados_limpio['sector'] == 'Pesqueras'].head(5)

,anio,mes,sector,empleados
1164,2001,enero,Pesqueras,353.0
1165,2001,febrero,Pesqueras,278.0
1166,2001,marzo,Pesqueras,275.0
1167,2001,abril,Pesqueras,273.0
1168,2001,mayo,Pesqueras,262.0


Imputar valores nulos de empleados en el sector Pesqueras usando el promedio del mes anterior y el posterior.

Diciembre 2001 → promedio entre noviembre 2001 y enero 2002

Diciembre 2002 → promedio entre noviembre 2002 y enero 2003

Diciembre 2004 → promedio entre noviembre 2004 y enero 2005



In [178]:
# Asegurarse de tener empleados como numérico
df_empleados_limpio['empleados'] = pd.to_numeric(df_empleados_limpio['empleados'], errors='coerce')

# Lista de casos a imputar manualmente
casos = [
    {'anio': 2001, 'mes': 'diciembre'},
    {'anio': 2002, 'mes': 'diciembre'},
    {'anio': 2004, 'mes': 'diciembre'}
]

for caso in casos:
    anio = caso['anio']
    mes = caso['mes']

    # Índice de la fila a reemplazar
    idx = df_empleados_limpio[
        (df_empleados_limpio['anio'] == anio) &
        (df_empleados_limpio['mes'] == mes) &
        (df_empleados_limpio['sector'] == 'Pesqueras')
    ].index

    if not idx.empty:
        i = idx[0]

        # Valor anterior y posterior
        empleados_ant = df_empleados_limpio.iloc[i - 1]['empleados']
        empleados_sig = df_empleados_limpio.iloc[i + 1]['empleados']

        # Promedio
        promedio = (empleados_ant + empleados_sig) / 2

        # Asignamos
        df_empleados_limpio.at[i, 'empleados'] = promedio


In [179]:
# Cuántos valores nulos hay por sector
df_empleados_limpio.isna().groupby(df_empleados_limpio['sector']).sum()

,anio,mes,sector,empleados
sector,,,,
Confeccionista,0,0,0,0
Electrónicas,0,0,0,0
Otras,0,0,0,0
Pesqueras,0,0,0,0
Plásticas,0,0,0,0
Textiles,0,0,0,0


In [180]:
df_empleados_limpio[
    df_empleados_limpio['sector'] == 'Pesqueras'].head(30)

,anio,mes,sector,empleados
1164,2001,enero,Pesqueras,353.0
1165,2001,febrero,Pesqueras,278.0
1166,2001,marzo,Pesqueras,275.0
1167,2001,abril,Pesqueras,273.0
1168,2001,mayo,Pesqueras,262.0
1169,2001,junio,Pesqueras,259.0
1170,2001,julio,Pesqueras,166.0
1171,2001,agosto,Pesqueras,166.0
1172,2001,septiembre,Pesqueras,188.0
1173,2001,octubre,Pesqueras,187.0


## DATASET DE ESTABLECIMIENTOS POR SECTOR INDUSTRIAL

In [181]:
# Cargar archivo
archivo_estab = ruta_base + '14_3_02_Establecimientos_industriales_rama-1.xlsx'
df_estab = pd.read_excel(archivo_estab, skiprows=3)

# Cortamos columnas de interés
df_estab = df_estab.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 9]]  
df_estab.columns = ['anio', 'mes', 'total', 'confeccionistas', 'electronicas',
                    'plasticas', 'textiles', 'pesqueras', 'otras']

# Rellenamos años faltantes
df_estab['anio'] = df_estab['anio'].ffill()

# Eliminamos filas completamente vacías
df_estab = df_estab.dropna(subset=['anio', 'mes'], how='all')

# Normalizar los valores de 'mes': minúsculas, sin espacios ni asteriscos
# Diccionario de normalización (corrige también abreviaturas y nombres mal escritos)
mes_completo = {
    'ene': 'enero', 'enero': 'enero',
    'feb': 'febrero', 'febrero': 'febrero',
    'mar': 'marzo', 'marzo': 'marzo',
    'abr': 'abril', 'abril': 'abril',
    'may': 'mayo', 'mayo': 'mayo',
    'jun': 'junio', 'junio': 'junio',
    'jul': 'julio', 'julio': 'julio',
    'ago': 'agosto', 'agosto': 'agosto',
    'sep': 'septiembre', 'septiembre': 'septiembre',
    'oct': 'octubre', 'octubre': 'octubre',
    'nov': 'noviembre', 'noviembre': 'noviembre',
    'dic': 'diciembre', 'diciembre': 'diciembre'
}

# Limpieza completa
df_empleados_limpio['mes'] = (
    df_empleados_limpio['mes']
    .astype(str)
    .str.strip()
    .str.lower()
    .str.replace(r'\*', '', regex=True)
    .map(mes_completo)  # ← reemplaza por nombres completos
)

# Reemplaza cualquier texto inválido como '- Cero absoluto' con NaN
df_estab['anio'] = pd.to_numeric(df_estab['anio'], errors='coerce')

# Rellena hacia abajo (por si año figura solo una vez cada 12 meses)
df_estab['anio'] = df_estab['anio'].ffill().astype(int)

# Eliminamos filas que no tienen mes (estas son las que generan los NaN al final)
df_estab = df_estab[df_estab['mes'].notna()]

# Pasamos a formato largo
df_estab_limpio = df_estab.melt(
    id_vars=['anio', 'mes'],
    var_name='sector',
    value_name='establecimientos'
)

# Capitalizamos nombre de sector
df_estab_limpio['sector'] = df_estab_limpio['sector'].str.capitalize()

# Mostrar primeras filas
print("✅ Dataset de establecimientos transformado:")
display(df_estab_limpio.head(15))


✅ Dataset de establecimientos transformado:


,anio,mes,sector,establecimientos
0,2001,Enero,Total,50.0
1,2001,Febrero,Total,47.0
2,2001,Marzo,Total,47.0
3,2001,Abril,Total,47.0
4,2001,Mayo,Total,47.0
5,2001,Junio,Total,47.0
6,2001,Julio,Total,47.0
7,2001,Agosto,Total,47.0
8,2001,Septiembre,Total,47.0
9,2001,Octubre,Total,47.0


In [182]:
print(sorted(df_estab_limpio['sector'].unique()))


['Confeccionistas', 'Electronicas', 'Otras', 'Pesqueras', 'Plasticas', 'Textiles', 'Total']


In [183]:
df_estab_limpio.isna().groupby(df_estab_limpio['sector']).sum()


,anio,mes,sector,establecimientos
sector,,,,
Confeccionistas,0,0,0,0
Electronicas,0,0,0,0
Otras,0,0,0,0
Pesqueras,0,0,0,0
Plasticas,0,0,0,0
Textiles,0,0,0,0
Total,0,0,0,0


In [184]:
# Filtramos el dataframe, excluyendo el sector 'Total'
df_estab_limpio = df_estab_limpio[df_estab_limpio['sector'].str.lower() != 'total']


In [185]:
display(df_estab_limpio.head(15))

,anio,mes,sector,establecimientos
291,2001,Enero,Confeccionistas,6.0
292,2001,Febrero,Confeccionistas,6.0
293,2001,Marzo,Confeccionistas,6.0
294,2001,Abril,Confeccionistas,6.0
295,2001,Mayo,Confeccionistas,6.0
296,2001,Junio,Confeccionistas,6.0
297,2001,Julio,Confeccionistas,6.0
298,2001,Agosto,Confeccionistas,6.0
299,2001,Septiembre,Confeccionistas,6.0
300,2001,Octubre,Confeccionistas,6.0


In [186]:
df_estab_limpio

,anio,mes,sector,establecimientos
291,2001,Enero,Confeccionistas,6.0
292,2001,Febrero,Confeccionistas,6.0
293,2001,Marzo,Confeccionistas,6.0
294,2001,Abril,Confeccionistas,6.0
295,2001,Mayo,Confeccionistas,6.0
...,...,...,...,...
2032,2024,Noviembre*,Otras,1.0
2033,2024,Diciembre*,Otras,1.0
2034,2025,Enero*,Otras,1.0
2035,2025,Febrero*,Otras,1.0


In [187]:
df_estab_limpio.isna().groupby(df_estab_limpio['sector']).sum()


,anio,mes,sector,establecimientos
sector,,,,
Confeccionistas,0,0,0,0
Electronicas,0,0,0,0
Otras,0,0,0,0
Pesqueras,0,0,0,0
Plasticas,0,0,0,0
Textiles,0,0,0,0


In [188]:
df_estab_limpio[df_estab_limpio['establecimientos'].isna()].sort_values(by=['sector', 'anio', 'mes'])


,anio,mes,sector,establecimientos


In [189]:
# Filtrar las filas válidas: que tengan mes no nulo
df_estab_limpio = df_estab_limpio[df_estab_limpio['mes'].notna()]

In [190]:
# Eliminar filas con 'mes' nulo → esas son las filas artificiales generadas al final
df_estab_limpio = df_estab_limpio[df_estab_limpio['mes'].notna()]

In [191]:
# Verificamos valores únicos
print(sorted(df_estab_limpio['mes'].unique()))

['Abril', 'Abril*', 'Agosto', 'Agosto*', 'Diciembre', 'Diciembre*', 'Enero ', 'Enero*', 'Febrero', 'Febrero*', 'Julio', 'Julio*', 'Junio', 'Junio*', 'Mar*', 'Marzo', 'Marzo*', 'Mayo', 'Mayo*', 'Noviembre', 'Noviembre*', 'Octubre', 'Octubre*', 'Septiembre', 'Septiembre*', 'abril', 'abril ', 'agosto', 'agosto ', 'diciembre', 'diciembre ', 'enero', 'enero ', 'febrero', 'febrero ', 'julio', 'julio ', 'junio', 'junio ', 'marzo', 'marzo ', 'mayo', 'mayo ', 'noviembre', 'noviembre ', 'octubre', 'octubre ', 'septiembre', 'septiembre ']


In [192]:
# Diccionario con los nombres limpios
meses_validos = {
    'ene': 'enero', 'feb': 'febrero', 'mar': 'marzo',
    'abr': 'abril', 'may': 'mayo', 'jun': 'junio',
    'jul': 'julio', 'ago': 'agosto', 'sep': 'septiembre',
    'oct': 'octubre', 'nov': 'noviembre', 'dic': 'diciembre'
}

# Limpieza unificada
df_estab_limpio['mes'] = (
    df_estab_limpio['mes']
    .astype(str)
    .str.lower()
    .str.strip()
    .str.replace(r'\*', '', regex=True)
    .str.replace(r'\s+', '', regex=True)  # quita espacios intermedios
    .map(lambda x: next((v for k, v in meses_validos.items() if x.startswith(k)), x))
)

In [193]:
# Verificamos valores únicos
print(sorted(df_estab_limpio['mes'].unique()))

['abril', 'agosto', 'diciembre', 'enero', 'febrero', 'julio', 'junio', 'marzo', 'mayo', 'noviembre', 'octubre', 'septiembre']


In [194]:
# Mostrar valores únicos en la columna 'establecimientos'
print(df_estab_limpio['establecimientos'].unique())


[ 6.  5.  7.  4.  3. 22. 21. 20. 17. 18. 16. 19. 23. 24. 25. 26. 27. 28.
  8.  9.  2.  1.  0.]


In [195]:
# Mostrar todas las filas donde la cantidad de establecimientos sea 0
print(df_estab_limpio[df_estab_limpio['establecimientos'] == 0])


      anio        mes     sector  establecimientos
1466  2001  diciembre  Pesqueras               0.0
1478  2002  diciembre  Pesqueras               0.0
1502  2004  diciembre  Pesqueras               0.0


In [196]:
# Reemplazar valores 0.0 por el promedio redondeado del mes anterior y siguiente

df_estab_limpio['establecimientos'] = df_estab_limpio['establecimientos'].astype(int)

# Reiniciamos el índice por seguridad
df_estab_limpio.reset_index(drop=True, inplace=True)

# Iteramos sobre los registros afectados
for idx in df_estab_limpio[
    (df_estab_limpio['sector'] == 'Pesqueras') & (df_estab_limpio['establecimientos'] == 0)
].index:
    
    try:
        val_ant = df_estab_limpio.loc[idx - 1, 'establecimientos']
        val_sig = df_estab_limpio.loc[idx + 1, 'establecimientos']
        
        if np.isfinite(val_ant) and np.isfinite(val_sig):
            promedio_redondeado = round((val_ant + val_sig) / 2)
            df_estab_limpio.at[idx, 'establecimientos'] = promedio_redondeado
    except:
        pass

# Verificamos que los cambios hayan sido aplicados correctamente
print(df_estab_limpio[
    (df_estab_limpio['sector'] == 'Pesqueras') &
    (df_estab_limpio['anio'].isin([2001, 2002, 2004])) &
    (df_estab_limpio['mes'] == 'diciembre')
])



      anio        mes     sector  establecimientos
1175  2001  diciembre  Pesqueras                 2
1187  2002  diciembre  Pesqueras                 2
1211  2004  diciembre  Pesqueras                 2


In [197]:
# Mostrar valores únicos en la columna 'establecimientos'
print(df_estab_limpio['establecimientos'].unique())


[ 6  5  7  4  3 22 21 20 17 18 16 19 23 24 25 26 27 28  8  9  2  1]


## VERIFICACION DE CORRECTA TRANSFORMACION DE DATASETS


In [198]:
# Función para revisar cualquier dataframe
def verificar_dataframe(df, nombre, valor_col):
    print(f"\n🔍 Verificando {nombre}:")
    print(f"Columnas: {df.columns.tolist()}")

    # Tipos de datos
    print("Tipos:")
    print(df.dtypes)

    # Nulos
    print("\n¿Nulos por columna?")
    print(df.isna().sum())

    # Valores únicos
    print("\nMeses únicos:", sorted(df['mes'].unique()))
    print("Sectores únicos:", sorted(df['sector'].unique()))

    # Ejemplo de filas
    print("\nPreview:")
    display(df.head(3))

# Llamadas por dataset
verificar_dataframe(df_produccion_total, "Producción", "Produccion")
verificar_dataframe(df_empleados_limpio, "Empleados", "empleados")
verificar_dataframe(df_estab_limpio, "Establecimientos", "establecimientos")



🔍 Verificando Producción:
Columnas: ['anio', 'mes', 'sector', 'Produccion']
Tipos:
anio           int32
mes           object
sector        object
Produccion    object
dtype: object

¿Nulos por columna?
anio          0
mes           0
sector        0
Produccion    0
dtype: int64

Meses únicos: ['abril', 'agosto', 'diciembre', 'enero', 'febrero', 'julio', 'junio', 'marzo', 'mayo', 'noviembre', 'octubre', 'septiembre']
Sectores únicos: ['Confeccionista', 'Electronica', 'Otros', 'Pesquera', 'Plastica', 'Textil']

Preview:


,anio,mes,sector,Produccion
0,2014,enero,Confeccionista,243751.0
1,2014,febrero,Confeccionista,386730.0
2,2014,marzo,Confeccionista,413565.0



🔍 Verificando Empleados:
Columnas: ['anio', 'mes', 'sector', 'empleados']
Tipos:
anio           int32
mes           object
sector        object
empleados    float64
dtype: object

¿Nulos por columna?
anio         0
mes          0
sector       0
empleados    0
dtype: int64

Meses únicos: ['abril', 'agosto', 'diciembre', 'enero', 'febrero', 'julio', 'junio', 'marzo', 'mayo', 'noviembre', 'octubre', 'septiembre']
Sectores únicos: ['Confeccionista', 'Electrónicas', 'Otras', 'Pesqueras', 'Plásticas', 'Textiles']

Preview:


,anio,mes,sector,empleados
0,2001,enero,Confeccionista,326.0
1,2001,febrero,Confeccionista,323.0
2,2001,marzo,Confeccionista,333.0



🔍 Verificando Establecimientos:
Columnas: ['anio', 'mes', 'sector', 'establecimientos']
Tipos:
anio                 int32
mes                 object
sector              object
establecimientos     int32
dtype: object

¿Nulos por columna?
anio                0
mes                 0
sector              0
establecimientos    0
dtype: int64

Meses únicos: ['abril', 'agosto', 'diciembre', 'enero', 'febrero', 'julio', 'junio', 'marzo', 'mayo', 'noviembre', 'octubre', 'septiembre']
Sectores únicos: ['Confeccionistas', 'Electronicas', 'Otras', 'Pesqueras', 'Plasticas', 'Textiles']

Preview:


,anio,mes,sector,establecimientos
0,2001,enero,Confeccionistas,6
1,2001,febrero,Confeccionistas,6
2,2001,marzo,Confeccionistas,6


In [199]:
print(sorted(df_empleados_limpio['sector'].unique()))
print(sorted(df_estab_limpio['sector'].unique()))
print(sorted(df_produccion_total['sector'].unique()))

['Confeccionista', 'Electrónicas', 'Otras', 'Pesqueras', 'Plásticas', 'Textiles']
['Confeccionistas', 'Electronicas', 'Otras', 'Pesqueras', 'Plasticas', 'Textiles']
['Confeccionista', 'Electronica', 'Otros', 'Pesquera', 'Plastica', 'Textil']


In [200]:
mapa_sectores = {
    'Confeccionista': 'Confeccionista',
    'Confeccionistas': 'Confeccionista',
    'Electrónica': 'Electronica',
    'Electrónicas': 'Electronica',
    'Electronicas': 'Electronica',
    'Pesquera': 'Pesquera',
    'Pesqueras': 'Pesquera',
    'Plástica': 'Plastica',
    'Plásticas': 'Plastica',
    'Plasticas': 'Plastica',
    'Textil': 'Textil',
    'Textiles': 'Textil',
    'Otro': 'Otros',
    'Otros': 'Otros',
    'Otras': 'Otros'
}

for df in [df_produccion_total, df_empleados_limpio, df_estab_limpio]:
    df['sector'] = df['sector'].replace(mapa_sectores)



In [201]:
print(sorted(df_empleados_limpio['sector'].unique()))
print(sorted(df_estab_limpio['sector'].unique()))
print(sorted(df_produccion_total['sector'].unique()))

['Confeccionista', 'Electronica', 'Otros', 'Pesquera', 'Plastica', 'Textil']
['Confeccionista', 'Electronica', 'Otros', 'Pesquera', 'Plastica', 'Textil']
['Confeccionista', 'Electronica', 'Otros', 'Pesquera', 'Plastica', 'Textil']


In [202]:
df_produccion_total['Produccion'] = pd.to_numeric(df_produccion_total['Produccion'], errors='coerce')

In [203]:
df_produccion_total[df_produccion_total['Produccion'].isna()]


,anio,mes,sector,Produccion
398,2024,septiembre,Pesquera,NaN
403,2025,febrero,Pesquera,NaN


In [204]:
df_produccion_total['Produccion'].isnull().sum()

2

In [205]:
# Ordenamos para asegurar que los reemplazos sean correctos
df_produccion_total = df_produccion_total.sort_values(by=['sector', 'anio', 'mes'])

# Reemplazamos ceros por NaN (opcional, si querés tratarlos igual que los nulos)
df_produccion_total['Produccion'] = df_produccion_total['Produccion'].replace(0, np.nan)

# Reemplazamos NaN con el valor anterior dentro del mismo sector
df_produccion_total['Produccion'] = df_produccion_total.groupby('sector')['Produccion'].fillna(method='ffill')


C:\Users\Luciano\AppData\Local\Temp\ipykernel_4868\3917298279.py:8: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  df_produccion_total['Produccion'] = df_produccion_total.groupby('sector')['Produccion'].fillna(method='ffill')
C:\Users\Luciano\AppData\Local\Temp\ipykernel_4868\3917298279.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_produccion_total['Produccion'] = df_produccion_total.groupby('sector')['Produccion'].fillna(method='ffill')


In [206]:
df_empleados_limpio['empleados'] = df_empleados_limpio['empleados'].astype(int)


In [207]:
df_estab_limpio['establecimientos'] = df_estab_limpio['establecimientos'].astype(int)


In [208]:
verificar_dataframe(df_produccion_total, "Producción", "Produccion")
verificar_dataframe(df_empleados_limpio, "Empleados", "empleados")
verificar_dataframe(df_estab_limpio, "Establecimientos", "establecimientos")


🔍 Verificando Producción:
Columnas: ['anio', 'mes', 'sector', 'Produccion']
Tipos:
anio            int32
mes            object
sector         object
Produccion    float64
dtype: object

¿Nulos por columna?
anio          0
mes           0
sector        0
Produccion    0
dtype: int64

Meses únicos: ['abril', 'agosto', 'diciembre', 'enero', 'febrero', 'julio', 'junio', 'marzo', 'mayo', 'noviembre', 'octubre', 'septiembre']
Sectores únicos: ['Confeccionista', 'Electronica', 'Otros', 'Pesquera', 'Plastica', 'Textil']

Preview:


,anio,mes,sector,Produccion
3,2014,abril,Confeccionista,446065.0
7,2014,agosto,Confeccionista,365418.0
11,2014,diciembre,Confeccionista,207726.0



🔍 Verificando Empleados:
Columnas: ['anio', 'mes', 'sector', 'empleados']
Tipos:
anio          int32
mes          object
sector       object
empleados     int32
dtype: object

¿Nulos por columna?
anio         0
mes          0
sector       0
empleados    0
dtype: int64

Meses únicos: ['abril', 'agosto', 'diciembre', 'enero', 'febrero', 'julio', 'junio', 'marzo', 'mayo', 'noviembre', 'octubre', 'septiembre']
Sectores únicos: ['Confeccionista', 'Electronica', 'Otros', 'Pesquera', 'Plastica', 'Textil']

Preview:


,anio,mes,sector,empleados
0,2001,enero,Confeccionista,326
1,2001,febrero,Confeccionista,323
2,2001,marzo,Confeccionista,333



🔍 Verificando Establecimientos:
Columnas: ['anio', 'mes', 'sector', 'establecimientos']
Tipos:
anio                 int32
mes                 object
sector              object
establecimientos     int32
dtype: object

¿Nulos por columna?
anio                0
mes                 0
sector              0
establecimientos    0
dtype: int64

Meses únicos: ['abril', 'agosto', 'diciembre', 'enero', 'febrero', 'julio', 'junio', 'marzo', 'mayo', 'noviembre', 'octubre', 'septiembre']
Sectores únicos: ['Confeccionista', 'Electronica', 'Otros', 'Pesquera', 'Plastica', 'Textil']

Preview:


,anio,mes,sector,establecimientos
0,2001,enero,Confeccionista,6
1,2001,febrero,Confeccionista,6
2,2001,marzo,Confeccionista,6


## UNIMOS LOS TRES DATASETS

In [218]:
# Unimos los datasets por anio, mes y sector
df_unificado = df_produccion_total.merge(
    df_empleados_limpio, on=['anio', 'mes', 'sector'], how='outer'
).merge(
    df_estab_limpio, on=['anio', 'mes', 'sector'], how='outer'
)

# Mostramos información general del nuevo dataframe
print("✅ Dataset unificado:")
display(df_unificado.head(10))
print("\n📐 Dimensiones:", df_unificado.shape)
print("\n🧪 Nulos por columna:\n", df_unificado.isna().sum())


✅ Dataset unificado:


,anio,mes,sector,Produccion,empleados,establecimientos
0,2001,abril,Confeccionista,NaN,337,6
1,2001,abril,Electronica,NaN,2249,20
2,2001,abril,Otros,NaN,103,4
3,2001,abril,Pesquera,NaN,273,1
4,2001,abril,Plastica,NaN,289,8
5,2001,abril,Textil,NaN,283,8
6,2001,agosto,Confeccionista,NaN,351,6
7,2001,agosto,Electronica,NaN,2637,20
8,2001,agosto,Otros,NaN,113,4
9,2001,agosto,Pesquera,NaN,166,1



📐 Dimensiones: (1746, 6)

🧪 Nulos por columna:
 anio                  0
mes                   0
sector                0
Produccion          937
empleados             0
establecimientos      0
dtype: int64


In [219]:
df_unificado

,anio,mes,sector,Produccion,empleados,establecimientos
0,2001,abril,Confeccionista,NaN,337,6
1,2001,abril,Electronica,NaN,2249,20
2,2001,abril,Otros,NaN,103,4
3,2001,abril,Pesquera,NaN,273,1
4,2001,abril,Plastica,NaN,289,8
...,...,...,...,...,...,...
1741,2025,marzo,Electronica,887363.0,6731,19
1742,2025,marzo,Otros,8844.0,435,1
1743,2025,marzo,Pesquera,513161.0,219,2
1744,2025,marzo,Plastica,NaN,652,7


In [220]:
orden_meses = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio',
               'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']

df_unificado['mes'] = pd.Categorical(df_unificado['mes'], categories=orden_meses, ordered=True)
df_unificado = df_unificado.sort_values(by=['anio', 'mes', 'sector']).reset_index(drop=True)


In [221]:
df_unificado.head(10)

,anio,mes,sector,Produccion,empleados,establecimientos
0,2001,enero,Confeccionista,NaN,326,6
1,2001,enero,Electronica,NaN,2652,22
2,2001,enero,Otros,NaN,90,4
3,2001,enero,Pesquera,NaN,353,2
4,2001,enero,Plastica,NaN,284,8
5,2001,enero,Textil,NaN,274,8
6,2001,febrero,Confeccionista,NaN,323,6
7,2001,febrero,Electronica,NaN,2624,21
8,2001,febrero,Otros,NaN,92,4
9,2001,febrero,Pesquera,NaN,278,1


“Hola profe, ya finalicé la integración y normalización de los tres datasets (producción, empleados y establecimientos) usando un merge por año, mes y sector. La base quedó unificada con 1746 registros, ordenada correctamente.

Quedan registros con producción nula en los años anteriores a 2013, ya que ese dataset no incluía datos antes de ese año.

¿Recomienda imputarlos, dejarlos como NaN o excluirlos en análisis futuros?

Exportar dataframes como csv

In [223]:
df_produccion_total.to_csv('../data/processed/produccion_total_por_sector.csv', index=False)

df_empleados_limpio.to_csv('../data/processed/empleados_total_por_sector.csv', index=False)

df_estab_limpio.to_csv('../data/processed/establecimientos_total_por_sector.csv', index=False)

df_unificado.to_csv('../data/processed/dataset_unificado_industria.csv', index=False)


Verifico la correcta exportacion

In [224]:
import os

print(os.listdir('../data/processed/'))


['.gitkeep', 'dataset_unificado_industria.csv', 'empleados_total_por_sector.csv', 'establecimientos_total_por_sector.csv', 'produccion_total_por_sector.csv']
